**In this project I used Python Pandas & Python Matplotlib to analyze and answer business questions about 12 months worth of sales data. The data contains hundreds of thousands of electronics store purchases broken down by month, product type, cost, purchase address, etc.**

# **IMPORTING LIBRARIES**

In [ ]:
import pandas as pd
import os         #The OS module in Python provides functions for interacting with the operating system.
import glob      #The glob module, which is short for global, is a function that's used to search for files that match a specific file pattern or name.
import matplotlib.pyplot as plt

**Task 1: Merge the 12 months of sales data into a single CSV file**

In [ ]:
files = [file for file in os.listdir('/kaggle/input/sales-datasets/')] #read single csv file

all_months_data = pd.DataFrame()                    #listed all files in a directory

for file in files:
    df = pd.read_csv("/kaggle/input/sales-datasets"+"/"+file)
    all_months_data = pd.concat([all_months_data,df])       #concatenated files
    
all_months_data.to_csv("all_data.csv", index = False)       #Converted files into csv format

In [ ]:
all_data = pd.read_csv("all_data.csv")       # read in updated dataframe
all_data.head()

In [ ]:
all_data.info()

# **DATA CLEANING**

**Clean up the NaN data**

In [ ]:
nan_df = all_data[all_data.isna().any(axis=1)]     #find NaN value from dataframe
nan_df.head()

In [ ]:
all_data = all_data.dropna(how = "all")      #removed NaN value
all_data.head()

**Find Or and delete it**

In [ ]:
all_data = all_data[all_data["Order Date"].str[0:2]!= "Or"] #removed the data containing "Or" in the "Order Date" column

**Convert column to the correct type**

In [ ]:
all_data["Quantity Ordered"] = pd.to_numeric(all_data["Quantity Ordered"]) # converted to int 
all_data["Price Each"] = pd.to_numeric(all_data["Price Each"])      # converted to float

# **AUGMENTED DATA WITH ADDITIONAL COLUMN AS PER THE REQUIREMENTS**

**Task 2: Add Month Column**

In [ ]:
all_data["Month"] = all_data["Order Date"].str[0:2]
all_data["Month"] = all_data["Month"].astype("int32")
all_data.head()

**Task 3: Add a sales column**

In [ ]:
all_data["Sales"] = all_data["Quantity Ordered"]*all_data["Price Each"]
all_data.head()

**Task 4: Add a city column**

In [ ]:
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

all_data["City"] = all_data["Purchase Address"].apply(lambda x:get_city(x) +' '+ get_state(x))
all_data.head()

# **EXPLORATORY DATA ANALYSIS**

# **Question 1: What was the best month for sales? how much was earned that month?**

In [ ]:
results = all_data.groupby("Month").sum()    #grouped by month 

In [ ]:
months = range(1,13)                         
plt.bar(months,results["Sales"])
plt.xticks(months)
plt.ylabel("Sales in USD ($)")
plt.xlabel("Month number")
plt.ticklabel_format(style='plain')
plt.show()

**Answer: December is the best month for sales and has earned more than 4 million usd**

# **Question 2: What city had the highest number of sales?**

In [ ]:
results = all_data.groupby("City").sum()

In [ ]:
cities = [city for city, df in all_data.groupby("City")]

plt.bar(cities,results["Sales"])
plt.xticks(cities, rotation = "vertical", size =8)
plt.ylabel("Sales in USD ($)")
plt.xlabel("City name")

plt.show()

**Answer: San Francisco has the highest number of sales**

# **Question 3: What time should we display advertisements to maximize likelihood of customers buying product?**

In [ ]:
all_data["Order Date"] = pd.to_datetime(all_data["Order Date"])   #converted Order Date to Datetime type

In [ ]:
all_data["Hour"] = all_data["Order Date"]. dt.hour      # extracted hour from Order Date
all_data["Minute"] = all_data["Order Date"]. dt.minute  # extracted minute from Order Date
all_data.head()

In [ ]:
hours = [Hour for Hour, df in all_data.groupby("Hour")]
plt.plot(hours, all_data.groupby(["Hour"]).count())
plt.xticks (hours)
plt.xlabel("Hour")
plt.ylabel("Number of Orders")
plt.grid()
plt.show()

**Answer: As per the graph time between 11 am to 1 pm and 6 pm to 8 pm will be the most suited for advertisement**

# **Question 4: What product are most often sold together?**

In [ ]:
all_data.head()

In [ ]:
df = all_data[all_data["Order ID"].duplicated(keep=False)]
df["Grouped"] = df.groupby("Order ID")["Product"].transform(lambda x:','.join(x))                                       
df.head()                          #created column grouped to keep the products sold together the most

In [ ]:
df = df[["Order ID", "Grouped"]].drop_duplicates()
df.head(100)

In [ ]:
from itertools import combinations     #This function prints all the possible combinations of the iterator with the specified length 
from collections import Counter        #This function is a sub-class that is used to count hashable objects

count = Counter()

for row in df["Grouped"]:
    row_list = row.split(",")
    count.update(Counter(combinations(row_list,2)))

for key, value in count.most_common(10):
    print(key, value)

**Answer: iPhone', 'Lightning Charging Cable are the products that are most often sold together**

# **Question 5: What product solds the most? why do you think it sold the most?**

In [ ]:
all_data.head()

In [ ]:
product_group = all_data.groupby("Product")
quantity_ordered  = product_group["Quantity Ordered"].sum()

products = [product for product, df in product_group]

plt.bar(products,quantity_ordered)
plt.ylabel("Quantity Ordered")
plt.xlabel("Product")
plt.xticks(products, rotation = "vertical", size = 8)
plt.show()                      # Below graph shows which product solds the most

**Answer: AAA Batteries (4-Pack) solds the most**

In [ ]:
prices = all_data.groupby("Product")["Price Each"].mean()

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(products, quantity_ordered, color = 'g')
ax2.plot(products, prices, 'b-')

ax1.set_xlabel('Product Name')
ax1.set_ylabel('Quantity Ordered', color = 'g')
ax2.set_ylabel('Price ($)', color='b')
ax1.set_xticklabels(products, rotation = "vertical", size = 8 )

plt.show()         #below graph shows why the particular product sold the most as per the price

**Answer: Due to the low price of AAA Batteries it has sold the most**